# 📄 AI-Powered PDF to DOCX Converter

## Overview
This notebook extracts text from PDF files using Google's Gemini AI and saves it as a clean, unformatted DOCX file.

---

## 🚀 Quick Start Guide

### Step 1: Get Your API Key
1. Visit [Google AI Studio](https://aistudio.google.com/)
2. Sign in with your Google account
3. Click "Get API Key" → "Create API Key"
4. Copy the generated key

### Step 2: Upload Your PDF
1. Click the folder icon in the left sidebar (Files panel)
2. Upload your PDF file to `/content/` or any directory

### Step 3: Run the Notebook
1. Run Cell 1 (Installation)
2. Run Cell 2 (Configuration & Core Functions)
3. Run Cell 3 (Interactive Form) - Fill in your details
4. Run Cell 4 (Execute Conversion)

---

## ⚙️ Configuration Options

| Parameter | Description | Default |
|-----------|-------------|---------|
| API Key | Your Gemini API key | Required |
| Model | AI model to use | gemini-2.5-flash-preview-05-20 |
| Input PDF | Path to source PDF | /content/sample.pdf |
| Output DOCX | Path for output file | /content/output.docx |

---

## 📊 Rate Limits (Free Tier)

| Model | RPM | TPM | RPD |
|-------|-----|-----|-----|
| Gemini 2.5 Flash | 10 | 250,000 | 500 |
| Gemini 1.5 Flash | 15 | 1,000,000 | 1,500 |

*RPM = Requests per minute, TPM = Tokens per minute, RPD = Requests per day*

---

## 🛠️ Troubleshooting

- **API Key Error**: Ensure your key is valid and has no extra spaces
- **File Not Found**: Check the file path and ensure the PDF is uploaded
- **Rate Limit**: Wait 60 seconds and retry, or upgrade to paid tier
- **Processing Failed**: Try a smaller PDF or different model

In [ ]:
# @title
# =============================================================================
# CELL 1: Installation
# =============================================================================
# Run this cell first to install required dependencies

!pip install python-docx google-generativeai ipywidgets --quiet
print("✅ Dependencies installed successfully!")

In [ ]:
# @title
# =============================================================================
# CELL 2: Configuration & Core Functions
# =============================================================================

import os
import time
import json
from pathlib import Path
from datetime import datetime, timedelta
from typing import Optional, Tuple, Dict, Any, List
from dataclasses import dataclass, field

try:
    from docx import Document
    import google.generativeai as genai
    from google.api_core import exceptions as google_exceptions
except ImportError as e:
    print(f"❌ Missing dependency: {e}")
    print("Please run the installation cell first!")
    raise

# Check if running in Colab
IN_COLAB = False
try:
    from google.colab import files as colab_files
    IN_COLAB = True
    print("✅ Running in Google Colab - file upload/download enabled")
except ImportError:
    print("ℹ️ Not running in Colab - using local file paths only")

# =============================================================================
# SIMPLE DATA CLASSES (NO ENUM FOR MODELS - USE STRINGS DIRECTLY)
# =============================================================================

@dataclass
class RateLimitConfig:
    """Configuration for rate limiting."""
    requests_per_minute: int = 15
    tokens_per_minute: int = 1_000_000
    requests_per_day: int = 1500
    request_timestamps: list = field(default_factory=list)
    daily_requests: int = 0
    last_reset_date: str = field(default_factory=lambda: datetime.now().strftime("%Y-%m-%d"))

@dataclass
class ConversionConfig:
    """Main configuration for the PDF to DOCX conversion."""
    api_key: str
    input_pdf: str
    output_docx: str
    model: str  # Just a string now - no enum!
    max_retries: int = 3
    retry_delay: int = 60

# =============================================================================
# MODEL DISCOVERY - FETCH REAL MODELS FROM API
# =============================================================================

class ModelDiscovery:
    """Discovers available models from the Gemini API."""

    _cached_models: List[str] = []
    _api_key: str = ""

    @classmethod
    def fetch_models(cls, api_key: str) -> List[str]:
        """
        Fetch available models from Gemini API.
        Returns list of model names that support generateContent.
        """
        if cls._api_key == api_key and cls._cached_models:
            return cls._cached_models

        cls._api_key = api_key
        cls._cached_models = []

        try:
            genai.configure(api_key=api_key)

            for model in genai.list_models():
                # Check if model supports content generation
                if hasattr(model, 'supported_generation_methods'):
                    methods = [str(m) for m in model.supported_generation_methods]
                    if 'generateContent' in methods:
                        # Extract clean model name
                        name = model.name.replace("models/", "")
                        cls._cached_models.append(name)

            # Sort to put preferred models first
            preferred_order = ['gemini-2.0-flash', 'gemini-1.5-flash', 'gemini-1.5-pro']

            def sort_key(model_name):
                for i, pref in enumerate(preferred_order):
                    if pref in model_name:
                        return (i, model_name)
                return (999, model_name)

            cls._cached_models.sort(key=sort_key)

            return cls._cached_models

        except Exception as e:
            print(f"⚠️ Error fetching models: {e}")
            # Return safe fallback
            return ["gemini-1.5-flash", "gemini-1.5-pro"]

    @classmethod
    def get_recommended_model(cls, api_key: str) -> str:
        """Get the recommended model from available ones."""
        models = cls.fetch_models(api_key)

        # Priority order
        for preferred in ["gemini-2.0-flash", "gemini-1.5-flash", "gemini-1.5-flash-8b"]:
            for model in models:
                if preferred in model:
                    return model

        return models[0] if models else "gemini-1.5-flash"

    @classmethod
    def clear_cache(cls):
        """Clear the cached models."""
        cls._cached_models = []
        cls._api_key = ""

# =============================================================================
# FILE HANDLER WITH RELIABLE DOWNLOAD
# =============================================================================

class FileHandler:
    """Handles file upload and download."""

    @staticmethod
    def upload_from_local() -> Optional[str]:
        """Upload a PDF file from local device (Colab only)."""
        if not IN_COLAB:
            print("⚠️ Local upload only available in Google Colab")
            return None

        print("📤 Please select a PDF file to upload...")
        try:
            uploaded = colab_files.upload()
            if not uploaded:
                print("❌ No file uploaded")
                return None

            filename = list(uploaded.keys())[0]
            if not filename.lower().endswith('.pdf'):
                print(f"❌ Error: {filename} is not a PDF file")
                return None

            filepath = f"/content/{filename}"
            print(f"✅ Uploaded: {filename} ({len(uploaded[filename]) / 1024:.1f} KB)")
            return filepath

        except Exception as e:
            print(f"❌ Upload failed: {e}")
            return None

    @staticmethod
    def download_file(filepath: str) -> bool:
        """
        Download a file. Returns True if successful.
        This is the RELIABLE method - uses direct Python call.
        """
        if not os.path.exists(filepath):
            print(f"❌ File not found: {filepath}")
            return False

        if not IN_COLAB:
            print(f"ℹ️ File saved to: {filepath}")
            print("   (Automatic download only available in Google Colab)")
            return True

        try:
            print(f"📥 Downloading {os.path.basename(filepath)}...")
            colab_files.download(filepath)
            print("✅ Download initiated! Check your browser's download folder.")
            return True
        except Exception as e:
            print(f"⚠️ Auto-download failed: {e}")
            print(f"\n📋 Manual download option:")
            print(f"   1. Click the folder icon (📁) in the left sidebar")
            print(f"   2. Navigate to: {filepath}")
            print(f"   3. Right-click → Download")
            return False

# =============================================================================
# RATE LIMITER
# =============================================================================

class RateLimiter:
    """Handles rate limiting for API calls."""

    def __init__(self):
        self.config = RateLimitConfig()
        self._reset_if_new_day()

    def _reset_if_new_day(self) -> None:
        today = datetime.now().strftime("%Y-%m-%d")
        if self.config.last_reset_date != today:
            self.config.daily_requests = 0
            self.config.last_reset_date = today

    def _clean_old_timestamps(self) -> None:
        cutoff = datetime.now() - timedelta(minutes=1)
        self.config.request_timestamps = [
            ts for ts in self.config.request_timestamps if ts > cutoff
        ]

    def wait_if_needed(self) -> None:
        self._reset_if_new_day()
        self._clean_old_timestamps()

        if len(self.config.request_timestamps) >= self.config.requests_per_minute:
            oldest = min(self.config.request_timestamps)
            wait_time = 60 - (datetime.now() - oldest).seconds + 5
            print(f"⏳ Rate limit reached. Waiting {wait_time} seconds...")
            time.sleep(wait_time)

    def record_request(self) -> None:
        self.config.request_timestamps.append(datetime.now())
        self.config.daily_requests += 1

# =============================================================================
# PDF CONVERTER (USES STRING MODEL NAME DIRECTLY)
# =============================================================================

class PDFConverter:
    """Main class for converting PDF to DOCX using AI."""

    def __init__(self, config: ConversionConfig):
        self.config = config
        self.rate_limiter = RateLimiter()
        genai.configure(api_key=config.api_key)

    def _validate_paths(self) -> None:
        if not os.path.exists(self.config.input_pdf):
            raise FileNotFoundError(f"Input PDF not found: {self.config.input_pdf}")

        if not self.config.input_pdf.lower().endswith('.pdf'):
            raise ValueError("Input file must be a PDF")

        output_dir = os.path.dirname(self.config.output_docx)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir, exist_ok=True)

    def _upload_to_gemini(self) -> Any:
        print(f"⬆️ Uploading {self.config.input_pdf}...")

        file = genai.upload_file(self.config.input_pdf, mime_type="application/pdf")
        print(f"   File '{file.display_name}' uploaded")

        max_wait = 300
        waited = 0

        while file.state.name == "PROCESSING":
            if waited >= max_wait:
                raise TimeoutError("File processing timed out")
            print(f"   Processing... ({waited}s)", end="\r")
            time.sleep(5)
            waited += 5
            file = genai.get_file(file.name)

        if file.state.name == "FAILED":
            raise ValueError("File processing failed on Gemini side")

        print(f"✅ File ready")
        return file

    def _extract_text(self, file_obj: Any) -> str:
        # Use the model string directly!
        model = genai.GenerativeModel(self.config.model)

        prompt = """
You are a highly accurate OCR and text extraction engine.

INSTRUCTIONS:
1. Extract ALL text from this document completely and accurately.
2. OUTPUT PLAIN TEXT ONLY - no markdown formatting.
3. Do NOT use: bold, italics, headers, bullet points, or any formatting.
4. Maintain paragraph structure with blank lines between paragraphs.
5. Preserve reading order and document flow.
6. Ignore repetitive headers, footers, and page numbers.

OUTPUT: Pure, clean, unformatted text only.
"""

        print(f"🧠 Analyzing document with {self.config.model}...")

        for attempt in range(self.config.max_retries):
            try:
                self.rate_limiter.wait_if_needed()
                self.rate_limiter.record_request()

                response = model.generate_content([file_obj, prompt])

                if response.text:
                    print("✅ Text extraction complete")
                    return response.text
                else:
                    raise ValueError("Empty response from AI model")

            except google_exceptions.ResourceExhausted as e:
                if attempt < self.config.max_retries - 1:
                    wait_time = self.config.retry_delay * (attempt + 1)
                    print(f"⚠️ Rate limit hit. Waiting {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    raise

            except google_exceptions.NotFound as e:
                print(f"❌ Model '{self.config.model}' not found!")
                raise

            except Exception as e:
                if attempt < self.config.max_retries - 1:
                    print(f"⚠️ Error: {str(e)}. Retrying...")
                    time.sleep(10)
                else:
                    raise

        raise Exception("Failed to extract text after all retries")

    def _create_docx(self, text: str) -> None:
        print(f"💾 Saving to {self.config.output_docx}...")

        doc = Document()
        lines = text.splitlines()
        current_paragraph = []

        for line in lines:
            stripped = line.strip()
            if stripped:
                current_paragraph.append(stripped)
            else:
                if current_paragraph:
                    doc.add_paragraph(' '.join(current_paragraph))
                    current_paragraph = []

        if current_paragraph:
            doc.add_paragraph(' '.join(current_paragraph))

        doc.save(self.config.output_docx)

        file_size = os.path.getsize(self.config.output_docx)
        size_str = f"{file_size / 1024:.1f} KB" if file_size > 1024 else f"{file_size} bytes"
        print(f"✨ Done! Output saved ({size_str})")

    def convert(self) -> str:
        print("\n" + "="*60)
        print("📄 PDF to DOCX Conversion")
        print("="*60)
        print(f"   Input:  {self.config.input_pdf}")
        print(f"   Output: {self.config.output_docx}")
        print(f"   Model:  {self.config.model}")
        print("="*60 + "\n")

        try:
            self._validate_paths()
            file_ref = self._upload_to_gemini()
            extracted_text = self._extract_text(file_ref)
            self._create_docx(extracted_text)

            print("\n" + "="*60)
            print("✅ CONVERSION SUCCESSFUL!")
            print("="*60 + "\n")

            return self.config.output_docx

        except Exception as e:
            print(f"\n❌ ERROR: {e}")
            raise

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def list_pdf_files(directory: str = "/content") -> list:
    """List all PDF files in a directory."""
    pdf_files = []
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.lower().endswith('.pdf'):
                pdf_files.append(os.path.join(directory, file))
    return pdf_files

print("✅ Core functions loaded successfully!")
print("\nℹ️ Models will be fetched dynamically when you enter your API key.")

In [ ]:
# @title
# =============================================================================
# CELL 3: Interactive Form - Fill in your details
# =============================================================================

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# =============================================================================
# STYLING
# =============================================================================

form_style = """
<style>
.form-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    border-radius: 10px 10px 0 0;
    margin-bottom: 0;
}
.section-box {
    border: 1px solid #ddd;
    padding: 15px;
    margin: 5px 0;
    border-radius: 5px;
    background: #fafafa;
}
.success-msg { color: #28a745; font-weight: bold; }
.error-msg { color: #dc3545; font-weight: bold; }
.info-msg { color: #17a2b8; }
</style>
"""

display(HTML(form_style))
display(HTML("<div class='form-header'><h2>🔧 Configuration Form</h2><p>Fill in the details below to convert your PDF</p></div>"))

# =============================================================================
# WIDGETS
# =============================================================================

# API Key Input
api_key_input = widgets.Password(
    value='',
    placeholder='Paste your Gemini API key here',
    description='API Key:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': '100px'}
)

api_key_status = widgets.HTML(value='<small>🔗 Get your API key from <a href="https://aistudio.google.com/" target="_blank">Google AI Studio</a></small>')

# Model dropdown - STARTS EMPTY, will be populated dynamically
model_dropdown = widgets.Dropdown(
    options=[('-- Enter API key first --', '')],
    value='',
    description='Model:',
    disabled=True,
    layout=widgets.Layout(width='100%'),
    style={'description_width': '100px'}
)

model_status = widgets.HTML(value='<small class="info-msg">ℹ️ Models will load after you validate your API key</small>')

# Load Models Button
load_models_btn = widgets.Button(
    description='🔑 Validate API Key & Load Models',
    button_style='warning',
    layout=widgets.Layout(width='250px')
)

models_output = widgets.Output()

def load_models_callback(b):
    """Validate API key and load available models dynamically."""
    with models_output:
        clear_output()

        api_key = api_key_input.value.strip()

        if not api_key:
            print("❌ Please enter your API key first")
            return

        if len(api_key) < 20:
            print("❌ API key appears too short")
            return

        print("🔄 Validating API key and fetching available models...")

        try:
            # Clear cache to force refresh
            ModelDiscovery.clear_cache()

            # Fetch models
            models = ModelDiscovery.fetch_models(api_key)

            if not models:
                print("❌ No models found or API key invalid")
                return

            # Create options for dropdown
            # Format: (display_name, value)
            model_options = []
            for m in models[:15]:  # Limit to first 15 models
                # Create friendly display name
                display_name = m.replace("-", " ").title()
                if "flash" in m.lower():
                    if "2.0" in m or "2-0" in m:
                        display_name = f"⚡ {m} (Fast, Latest)"
                    elif "1.5" in m:
                        display_name = f"⚡ {m} (Recommended)"
                elif "pro" in m.lower():
                    display_name = f"🎯 {m} (High Quality)"
                else:
                    display_name = f"📄 {m}"

                model_options.append((display_name, m))

            # Update dropdown
            model_dropdown.options = model_options
            model_dropdown.value = models[0]  # Select first (best) model
            model_dropdown.disabled = False

            print(f"✅ API key valid! Found {len(models)} available models.")
            print(f"📋 Selected: {models[0]}")

            model_status.value = f'<small class="success-msg">✅ {len(model_options)} models loaded. Selected: {models[0]}</small>'
            api_key_status.value = '<small class="success-msg">✅ API key validated successfully</small>'

        except Exception as e:
            print(f"❌ Error: {e}")
            model_dropdown.options = [('-- Error loading models --', '')]
            model_dropdown.disabled = True

load_models_btn.on_click(load_models_callback)

# =============================================================================
# FILE INPUT SECTION
# =============================================================================

pdf_path_input = widgets.Text(
    value='/content/sample.pdf',
    placeholder='/content/your_file.pdf',
    description='Input PDF:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': '100px'}
)

pdf_list_output = widgets.HTML(value='<small>Click buttons below to find or upload PDFs</small>')

def scan_for_pdfs(b):
    pdfs = list_pdf_files('/content')
    if pdfs:
        pdf_list_output.value = f"<small>📁 Found: {', '.join([os.path.basename(p) for p in pdfs])}</small>"
        pdf_path_input.value = pdfs[0]
    else:
        pdf_list_output.value = "<small>⚠️ No PDF files found in /content</small>"

scan_button = widgets.Button(description='🔍 Scan /content', button_style='info', layout=widgets.Layout(width='140px'))
scan_button.on_click(scan_for_pdfs)

upload_output = widgets.Output()

def upload_from_computer(b):
    with upload_output:
        clear_output()
        if IN_COLAB:
            uploaded_path = FileHandler.upload_from_local()
            if uploaded_path:
                pdf_path_input.value = uploaded_path
                pdf_list_output.value = f"<small class='success-msg'>✅ Uploaded: {os.path.basename(uploaded_path)}</small>"
        else:
            print("⚠️ Upload only works in Google Colab")

upload_button = widgets.Button(description='📤 Upload PDF', button_style='success', layout=widgets.Layout(width='140px'))
upload_button.on_click(upload_from_computer)

# =============================================================================
# OUTPUT SETTINGS
# =============================================================================

output_path_input = widgets.Text(
    value='/content/output.docx',
    placeholder='/content/output.docx',
    description='Output:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': '100px'}
)

auto_name_checkbox = widgets.Checkbox(
    value=True,
    description='Auto-generate output filename',
    layout=widgets.Layout(width='100%')
)

def update_output_path(change):
    if auto_name_checkbox.value and pdf_path_input.value:
        base = os.path.splitext(pdf_path_input.value)[0]
        output_path_input.value = f"{base}_converted.docx"

pdf_path_input.observe(update_output_path, names='value')
auto_name_checkbox.observe(update_output_path, names='value')

# =============================================================================
# FINAL VALIDATION
# =============================================================================

status_output = widgets.Output()

def validate_and_save(b):
    with status_output:
        clear_output()

        errors = []

        # Check API key
        if not api_key_input.value.strip():
            errors.append("❌ API Key is required")
        elif len(api_key_input.value.strip()) < 20:
            errors.append("❌ API Key appears invalid")

        # Check model selected
        if not model_dropdown.value:
            errors.append("❌ Please validate API key and select a model first")

        # Check input file
        if not pdf_path_input.value.strip():
            errors.append("❌ Input PDF path is required")
        elif not os.path.exists(pdf_path_input.value):
            errors.append(f"❌ File not found: {pdf_path_input.value}")

        # Check output path
        if not output_path_input.value.strip():
            errors.append("❌ Output path is required")
        elif not output_path_input.value.endswith('.docx'):
            errors.append("❌ Output must be a .docx file")

        if errors:
            for error in errors:
                print(error)
            print("\n⚠️ Please fix errors above.")
        else:
            print("✅ All settings validated!")
            print("\n📋 Configuration:")
            print(f"   • Model: {model_dropdown.value}")
            print(f"   • Input: {pdf_path_input.value}")
            print(f"   • Output: {output_path_input.value}")
            print("\n➡️ Run the next cell (Cell 5) to start conversion!")

            # Store config globally
            global USER_CONFIG
            USER_CONFIG = {
                'api_key': api_key_input.value.strip(),
                'model': model_dropdown.value,  # Direct string, no enum!
                'input_pdf': pdf_path_input.value.strip(),
                'output_docx': output_path_input.value.strip()
            }

validate_button = widgets.Button(
    description='✓ Save Configuration',
    button_style='success',
    layout=widgets.Layout(width='200px')
)
validate_button.on_click(validate_and_save)

# =============================================================================
# LAYOUT
# =============================================================================

# Section 1: API Key & Model (THE KEY IMPROVEMENT!)
api_section = widgets.VBox([
    widgets.HTML("<h4>🔑 Step 1: API Key & Model Selection</h4>"),
    api_key_input,
    api_key_status,
    widgets.HTML("<br>"),
    load_models_btn,  # <-- NEW: Button to load models
    models_output,
    widgets.HTML("<br>"),
    model_dropdown,
    model_status,
], layout=widgets.Layout(padding='15px', border='1px solid #ddd', margin='5px 0'))

# Section 2: File Selection
file_section = widgets.VBox([
    widgets.HTML("<h4>📁 Step 2: Select PDF File</h4>"),
    pdf_path_input,
    widgets.HBox([scan_button, upload_button]),
    pdf_list_output,
    upload_output,
], layout=widgets.Layout(padding='15px', border='1px solid #ddd', margin='5px 0'))

# Section 3: Output Settings
output_section = widgets.VBox([
    widgets.HTML("<h4>💾 Step 3: Output Settings</h4>"),
    output_path_input,
    auto_name_checkbox,
], layout=widgets.Layout(padding='15px', border='1px solid #ddd', margin='5px 0'))

# Section 4: Validate & Run
action_section = widgets.VBox([
    widgets.HTML("<h4>🚀 Step 4: Validate & Run</h4>"),
    validate_button,
    status_output
], layout=widgets.Layout(padding='15px', border='1px solid #ddd', margin='5px 0'))

# Display
display(api_section)
display(file_section)
display(output_section)
display(action_section)

# Auto-scan for PDFs
scan_for_pdfs(None)

In [ ]:
# @title
# =============================================================================
# CELL 4: Execute Conversion
# =============================================================================

def run_conversion():
    """Execute the PDF to DOCX conversion."""

    # Check if configuration exists
    if 'USER_CONFIG' not in globals():
        print("❌ Configuration not found!")
        print("   Please run Cell 4 and click 'Save Configuration'")
        return None

    config = USER_CONFIG

    # Validate model is selected
    if not config.get('model'):
        print("❌ No model selected!")
        print("   Please validate your API key in Cell 4 first.")
        return None

    # Create conversion config (using string model directly!)
    conversion_config = ConversionConfig(
        api_key=config['api_key'],
        input_pdf=config['input_pdf'],
        output_docx=config['output_docx'],
        model=config['model'],  # Direct string - no enum conversion!
        max_retries=3,
        retry_delay=60
    )

    # Run the conversion
    try:
        converter = PDFConverter(conversion_config)
        output_path = converter.convert()

        print("\n" + "="*60)
        print("📥 DOWNLOAD YOUR FILE")
        print("="*60)

        # Use the reliable download method
        download_success = FileHandler.download_file(output_path)

        if not download_success and IN_COLAB:
            # Provide backup instructions
            print("\n" + "-"*60)
            print("📋 BACKUP: Manual Download Instructions")
            print("-"*60)
            print("1. Click the 📁 folder icon in the left sidebar")
            print(f"2. Find: {output_path}")
            print("3. Right-click → Download")

        return output_path

    except Exception as e:
        print(f"\n❌ Conversion failed: {str(e)}")
        print("\n💡 Troubleshooting:")
        print("   1. Verify your API key is valid")
        print("   2. Check the PDF file exists")
        print("   3. Try a different model from the dropdown")
        return None

# =============================================================================
# RUN
# =============================================================================

print("🚀 Starting conversion...\n")
output_file = run_conversion()

# =============================================================================
# SEPARATE DOWNLOAD CELL (BACKUP)
# =============================================================================

if output_file:
    print("\n" + "="*60)
    print("💡 IF DOWNLOAD DIDN'T START AUTOMATICALLY:")
    print("="*60)
    print("Run this code in a new cell:")
    print(f"\n    from google.colab import files")
    print(f"    files.download('{output_file}')")